In [1]:
from bettmensch_ai.pipelines import OutputArtifact, InputArtifact
import torch
from bettmensch_ai_examples.gpt_1.src.train import pretrain, GPTPretrainConfig, GPTTrainer, TokenizedBookCorpusOpenSplit
from bettmensch_ai_examples.gpt_1.component import get_source_data_split, get_tokenized_data_split_and_tokenizer, pretrain_and_checkpoint

W0301 01:48:55.898000 18592 torch\distributed\elastic\multiprocessing\redirects.py:27] NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\scher\Repositories\bettmensch.ai-examples\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Pipeline pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pipelines.pipeline_context.PipelineContext object at 0x0000029D5027EBD0>
Base Component pipeline context: <bettmensch_ai.pi

In [2]:
get_source_data = False

if get_source_data:
    get_source_data_split(
        data_out=OutputArtifact(name="source_data_train")
    )

In [3]:
preprocess_data = False
N_TOTAL = 74004228
train_split = 0.9
n_train_observations = int(train_split * N_TOTAL)
sequence_length_test = 128

if preprocess_data:
    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        end_index=n_train_observations,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_train"),
        tokenizer_out=OutputArtifact(name="tokenizer_train")
    )

    get_tokenized_data_split_and_tokenizer(
        source_data_split=InputArtifact(name="source_data_train"),
        start_index=n_train_observations,
        end_index=-1,
        sequence_length=sequence_length_test,
        tokenized_data_out=OutputArtifact(name="tokenized_data_validation"),
        tokenizer_out=OutputArtifact(name="tokenizer_validation")
    )

In [4]:
config_test = GPTPretrainConfig.from_file("C:\\Users\\scher\\Repositories\\bettmensch.ai-examples\\sdk\\bettmensch_ai_examples\\gpt_1\\src\\config.yaml")
print(config_test.data)
print(config_test.tokenizer)
print(config_test.model)
print(config_test.trainer)

{'train': {'dataset': {'path': 'tokenized_data_validation', 'context_size': 128}, 'dataloader': {'batch_size': 45, 'shuffle': True}}, 'validation': {'use': True, 'dataset': {'path': 'tokenized_data_validation', 'context_size': 128}, 'dataloader': {'batch_size': 80, 'shuffle': False}}}
{'path': 'tokenizer_train'}
{'architecture': {'n_tokens': 128, 'dim_embed': 768, 'n_decoder_layers': 12, 'n_heads': 12, 'dropout': 0.1}, 'misc': {'verbose': False}}
{'optimizer': {'lr': 0.00025, 'weight_decay': 0.01, 'betas': [0.9, 0.95], 'eps': 0.0001}, 'scheduler': {'linear': {'start_factor': 0.0001, 'end_factor': 1, 'total_iters': 2000}, 'cosine': {'T_max': 500, 'eta_min': 0}, 'sequential': {'milestones': [2000]}}, 'training': {'grad_clip_norm': 1.0, 'n_steps': 1000000, 'eval_steps': 50, 'n_steps_eval': 10, 'checkpoint_steps': 7500}}


In [ ]:
train = True

if train:
    pretrain(config_test)

2025-03-01 01:50:18 | Step 50/1000000 | Train loss: 10.40625 | Validation loss: 11.0 | Learning rate: 6.274375000000002e-06
2025-03-01 01:51:19 | Step 100/1000000 | Train loss: 10.1125 | Validation loss: 10.83125 | Learning rate: 1.252375e-05


In [ ]:
resume_training = False

if resume_training:
    torch.set_default_device("cuda")
    train_data = TokenizedBookCorpusOpenSplit(**config_test.data['train']['dataset'])

    train_loader = torch.utils.data.DataLoader(
        train_data, collate_fn=TokenizedBookCorpusOpenSplit.collate_batch, **config_test.data['train']['dataloader'],generator=torch.Generator(device="cuda")
    )
    torch.set_default_device("cuda")
    trainer = GPTTrainer.from_checkpoint(path="", train_loader=train_loader)
    trainer.train()

EPOCH 5:
2025-02-28 10:36:41 | Batch 10/18424 (observations 405-450/829080) loss: 4.1578125
2025-02-28 10:37:00 | Batch 20/18424 (observations 855-900/829080) loss: 4.18125
2025-02-28 10:37:18 | Batch 30/18424 (observations 1305-1350/829080) loss: 4.0984375
2025-02-28 10:37:37 | Batch 40/18424 (observations 1755-1800/829080) loss: 4.115625
2025-02-28 10:37:56 | Batch 50/18424 (observations 2205-2250/829080) loss: 4.146875
2025-02-28 10:38:14 | Batch 60/18424 (observations 2655-2700/829080) loss: 4.134375
2025-02-28 10:38:32 | Batch 70/18424 (observations 3105-3150/829080) loss: 4.134375
2025-02-28 10:38:51 | Batch 80/18424 (observations 3555-3600/829080) loss: 4.18125
2025-02-28 10:39:09 | Batch 90/18424 (observations 4005-4050/829080) loss: 4.1375
2025-02-28 10:39:28 | Batch 100/18424 (observations 4455-4500/829080) loss: 4.1140625
2025-02-28 10:39:46 | Batch 110/18424 (observations 4905-4950/829080) loss: 4.14375
2025-02-28 10:40:05 | Batch 120/18424 (observations 5355-5400/829080) l

KeyboardInterrupt: 